In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import os

In [2]:
# copied from https://github.com/benmoseley/harmonic-oscillator-pinn/blob/main/Harmonic%20oscillator%20PINN.ipynb
class FCN(torch.nn.Module):    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = torch.nn.Tanh
        
        self.fcs = torch.nn.Sequential(*[
                        torch.nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = torch.nn.Sequential(*[
                        torch.nn.Sequential(*[
                            torch.nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = torch.nn.Linear(N_HIDDEN, N_OUTPUT)

      # Apply custom weight initialization
        self.apply(self._init_weights)
    
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        act = torch.nn.Softplus()
        x = act(x)
        return x

    def _init_weights(self, m):
        if isinstance(m, torch.nn.Linear):
            # Get the size of the previous layer (input size of the current layer)
            n = m.in_features # The number of input features to this layer
            # Set the range for uniform distribution as [-1/sqrt(n), 1/sqrt(n)]
            bound = 1 / np.sqrt(n)
            # Initialize weights with a uniform distribution in the range [-bound, bound]
            torch.nn.init.uniform_(m.weight, -bound, bound)
            
            # Initialize biases to zero, only if the layer has biases
            if m.bias is not None:
                torch.nn.init.constant_(m.bias, 0.0)

In [3]:
# copied from https://github.com/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb
def gradients(outputs, inputs, order = 1):
    if order == 1:
        return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)[0]
    elif order > 1:
        return gradients(gradients(outputs, inputs, 1), inputs, order - 1)
    else:
        return outputs

In [4]:
# random (uniform) sample points in (0,1).
def random_domain_points(n):
    #x = torch.rand((n,1), requires_grad=True)
    xhigh = 0.5*torch.rand((int(n/2),1), requires_grad=True) + 0.5 # [0.5,1)
    xlow  = -0.5*torch.rand((int(n/2),1), requires_grad=True) + 0.5 # (0,0.5]
    x = torch.cat((xlow, xhigh),0)
    return x

In [9]:
device = torch.device("cpu")

dir = "../models/GPU_non_compact_r_fixed_omega/"
dir = dir + "neurons256_h_layers4_rmax1000_n2000_sigma0.1/"

neurons = 256
layers = 4

model = FCN(1,4,neurons, layers)#.to(device)


In [ ]:
# test loss

# 1. Load your model and switch to train mode (to enable gradients)
model.load_state_dict(torch.load(dir + "model_epoch200000.pth", map_location=device))
model.train()  # important for gradient computation

# 2. Prepare input tensor with requires_grad=True
# Example for 1D input (time t)
x = torch.linspace(0, 1, 100).unsqueeze(1)  # shape (100, 1)
x.requires_grad_(True)

# 3. Forward pass
u = model(x)

# 4. Define your loss (example: dummy loss, replace with your residual loss)
# For example, if your ODE residual is R(u, x), define:
# loss = torch.mean(R(u, x)**2)
# Here, we just use a dummy loss: mean squared of u to keep it simple
loss = torch.mean(u**2)

# 5. Zero existing gradients (good practice)
model.zero_grad()

# 6. Backward pass
loss.backward()

# 7. Check and print gradient norms for all parameters
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name}: grad norm = {param.grad.norm().item():.2e}")
    else:
        print(f"{name}: no grad")


fcs.0.weight: grad norm = 1.01e-03
fcs.0.bias: grad norm = 1.35e-03
fch.0.0.weight: grad norm = 2.49e-02
fch.0.0.bias: grad norm = 1.63e-03
fch.1.0.weight: grad norm = 2.89e-02
fch.1.0.bias: grad norm = 1.93e-03
fch.2.0.weight: grad norm = 7.72e-02
fch.2.0.bias: grad norm = 4.94e-03
fce.weight: grad norm = 6.01e+00
fce.bias: grad norm = 3.85e-01


/tmp/ipykernel_12677/2763666089.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(dir + "model_epoch200000.pth", map_location=device))


In [13]:
for name, param in model.named_parameters():
    print(f"{name}: param = {param.grad}")

fcs.0.weight: param = tensor([[-6.7740e-10],
        [ 1.9592e-07],
        [ 3.2176e-08],
        [-4.2725e-06],
        [ 3.1326e-09],
        [-1.5827e-07],
        [-5.9647e-08],
        [ 2.4378e-07],
        [-2.4818e-09],
        [-2.2968e-06],
        [-1.3508e-08],
        [-1.8403e-08],
        [ 2.7657e-09],
        [-8.9091e-07],
        [-3.2996e-06],
        [ 2.2053e-09],
        [ 2.5121e-08],
        [-1.6397e-08],
        [ 8.8085e-08],
        [-6.9442e-07],
        [-4.0941e-10],
        [-3.5903e-07],
        [-4.6792e-09],
        [-3.0208e-06],
        [ 1.7567e-06],
        [-7.4281e-08],
        [-7.1025e-09],
        [-5.4649e-10],
        [-1.7954e-09],
        [ 9.6616e-08],
        [-2.3885e-06],
        [-1.0222e-07],
        [-3.6752e-09],
        [-3.2044e-05],
        [ 2.9035e-07],
        [ 2.7330e-09],
        [ 4.4044e-07],
        [ 5.0226e-07],
        [-3.1213e-04],
        [ 2.7047e-04],
        [-7.3231e-05],
        [ 1.5957e-08],
        [ 1.

In [15]:
# Assume: loss_f1, loss_f2, loss_f3, loss_f4 represent your 4 ODE residual losses
# These should be computed from the same forward pass if possible
# If your model outputs a vector [u1, u2, u3, u4], you can separate them as needed

# Step 1: Load model
model.load_state_dict(torch.load(dir + "model_epoch200000.pth", map_location=device))
model.train()

# Step 2: Input (enable gradients for PINN-style derivatives)
r = torch.linspace(0, 30, 1000).unsqueeze(1).to(device).requires_grad_(True)

# Step 3: Forward pass
u = model(r)  # assume u has shape (100, 4)

# Step 4: Split u if needed
A, alpha, chi_minus, phi = map(lambda i:  u[:, [i]], range(4))
chi = - chi_minus
Ar = gradients(A, r)
alphar = gradients(alpha, r)
chir = gradients(chi, r)
phir = gradients(phi, r)

omega = 0.895042 * torch.ones(1).to(device)
phi0  = 0.05  * torch.ones(1).to(device)
m = torch.ones(1).to(device)

V = 0.5 * torch.pow(m, 2) * torch.pow(phi, 2)
dVdphi = torch.pow(m, 2) * phi
rho = 0.5 * (torch.pow(chi, 2) / A + torch.pow((omega / alpha), 2) * torch.pow(phi, 2)) + V
SA = 0.5 * (torch.pow(chi, 2) / A + torch.pow((omega / alpha), 2) * torch.pow(phi, 2)) - V
eq_A = r * Ar - A * ((1 - A) + 8 * torch.pi * (r ** 2) * A * rho)
eq_alpha = r * alphar - alpha * (0.5 * (A - 1) + 8 * torch.pi * A * (r ** 2) * SA)
eq_chi = r * chir + (chi) * (1 + A - 8 * torch.pi * r * A * V) - r * A * (dVdphi - torch.pow((omega / alpha), 2) * phi)
eq_phi = phir - chi

# Step 5: Define your four individual losses (these are just placeholders)
# Replace these with your actual physics-informed residuals
loss_f1 = torch.mean(torch.pow(eq_A, 2))
loss_f2 = torch.mean(torch.pow(eq_alpha, 2))
loss_f3 = torch.mean(torch.pow(eq_chi, 2))
loss_f4 = torch.mean(torch.pow(eq_phi, 2))

# Step 6: For each loss, backward and check grads
losses = [loss_f1, loss_f2, loss_f3, loss_f4]
loss_names = ['loss_f1', 'loss_f2', 'loss_f3', 'loss_f4']

for i, (loss, name) in enumerate(zip(losses, loss_names)):
    model.zero_grad()  # Clear gradients before each backward
    loss.backward(retain_graph=True)  # retain_graph=True allows reuse of computation
    print(f"\n{name} gradients:")
    for pname, param in model.named_parameters():
        if param.grad is not None:
            print(f"  {pname}: grad norm = {param.grad.norm().item():.2e}")
        else:
            print(f"  {pname}: no grad")


/tmp/ipykernel_12677/4022050355.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(dir + "model_epoch200000.pth", map_location=device))



loss_f1 gradients:
  fcs.0.weight: grad norm = 1.61e-04
  fcs.0.bias: grad norm = 3.37e-05
  fch.0.0.weight: grad norm = 2.62e-04
  fch.0.0.bias: grad norm = 1.71e-05
  fch.1.0.weight: grad norm = 3.50e-04
  fch.1.0.bias: grad norm = 2.43e-05
  fch.2.0.weight: grad norm = 5.26e-03
  fch.2.0.bias: grad norm = 3.38e-04
  fce.weight: grad norm = 1.12e-02
  fce.bias: grad norm = 7.08e-04

loss_f2 gradients:
  fcs.0.weight: grad norm = 5.31e-05
  fcs.0.bias: grad norm = 4.93e-06
  fch.0.0.weight: grad norm = 6.88e-05
  fch.0.0.bias: grad norm = 4.43e-06
  fch.1.0.weight: grad norm = 6.58e-05
  fch.1.0.bias: grad norm = 4.65e-06
  fch.2.0.weight: grad norm = 3.24e-04
  fch.2.0.bias: grad norm = 2.05e-05
  fce.weight: grad norm = 1.90e-03
  fce.bias: grad norm = 1.25e-04

loss_f3 gradients:
  fcs.0.weight: grad norm = 1.55e-06
  fcs.0.bias: grad norm = 1.75e-07
  fch.0.0.weight: grad norm = 3.65e-06
  fch.0.0.bias: grad norm = 2.34e-07
  fch.1.0.weight: grad norm = 3.47e-06
  fch.1.0.bias: g